In [ ]:
!pip install torch transformers -U -q accelerate -U protobuf scipy utils datasets pandas matplotlib seaborn gdown sentencepiece nltk evaluate rouge_score
!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U datasets

In [ ]:
from transformers import AutoTokenizer, LlamaForCausalLM
import numpy as np
import torch
from torch import nn

import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
from utils import *


# from google.colab import drive
import pandas as pd

import datasets

# # initialize the model
# model_path = "Phind/Phind-CodeLlama-34B-v2"
# model = LlamaForCausalLM.from_pretrained(model_path, device_map="auto")
# tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
prompt = '''
### System Prompt
You are an intelligent programming Tester.

### User Message
Generate test cases for the given functions written in C such that every line of code will be excuted once at least to achieve coverging 100%
Consider all possible combinations for the set of inputs

Example:
    Input sample:
      // A function that calculates the factorial of a positive integer
      int factorial(int n) {
        // Check if n is valid
        if (n < 0) {
          return -1;
        }
        // Base case
        if (n == 0 || n == 1) {
          return 1;
        }
        // Recursive case
        return n * factorial(n - 1);
      }

in the previous example the only input is intgre so the possible combinations is zero or positive integre or negative one
so the following test cases cover all these combinations

    Corresponding Output:
      {
        "test_case 1": [-1],
        "test_case 2": [-5],
        "test_case 3": [0],
        "test_case 4": [1],
        "test_case 5": [5]
      }

### Assistant
// A function that reverses a string
char* reverse_string(char* str) {
  // Get the length of the string
  int len = strlen(str);
  // Allocate memory for the reversed string
  char* rev = malloc(len + 1);
  // Check if the memory allocation was successful
  if (rev == NULL) {
    return NULL;
  }
  // Copy the characters from the original string to the reversed string in reverse order
  for (int i = 0; i < len; i++) {
    rev[i] = str[len - i - 1];
  }
  // Add the null terminator to the reversed string
  rev[len] = '\0';
  // Return the reversed string
  return rev;
}
'''

In [ ]:

def generate_one_completion(prompt: str):
    tokenizer.pad_token_id = tokenizer.eos_token_id
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096)

    # Generate
    generate_ids = model.generate(inputs.input_ids.to("cuda"), max_new_tokens=384, do_sample=True, top_p=0.75, top_k=40, temperature=0.1)
    completion = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    completion = completion.replace(prompt, "").split("\n\n\n")[0]

    return completion
test_cases = generate_one_completion(prompt)
print(test_cases)

In [ ]:
next(model.parameters()).device

device(type='cuda', index=0)

In [ ]:
# Import the tokenizer and the model
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer2 = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True)
model2 = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True).cuda()

# Define a prompt that instructs the model to generate test cases for the factorial function
prompt2 = """
Write test cases for the following function written in C:

// A function that calculates the factorial of a positive integer
int factorial(int n) {
  // Check if n is valid
  if (n < 0) {
    return -1;
  }
  // Base case
  if (n == 0 || n == 1) {
    return 1;
  }
  // Recursive case
  return n * factorial(n - 1);
}
"""


KeyboardInterrupt



In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True).cuda()
# messages = [
#     { 'role': 'user', 'content': "write a quick sort algorithm in python."}
# ]
# inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)
# # 32021 is the id of <|EOT|> token
# outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=32021)
# print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

In [ ]:
# Use the tokenizer to encode the prompt
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096)

# Use the model to generate text
outputs = model2.generate(inputs.input_ids.to("cuda"), max_new_tokens=384, do_sample=True, top_p=0.75, top_k=40, temperature=0.1)

# Use the tokenizer to decode the text
text = tokenizer2.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

# Print the generated text
print(text)

In [ ]:
import json

# hn7ot el output bta3 el model
json_string = '''
{
  "test_case 1": ["Hello"],
  "test_case 2": [""],
  "test_case 3": ["  Hello World  "],
  "test_case 4": ["!@#$%^&*()"],
  "test_case 5": ["12345"]
}
'''
data = json.loads(json_string)

with open('output_parsing.txt', 'w') as txt_file:
    for key, values in data.items():
        txt_file.write(' '.join(map(str, values)) + '\n')

In [ ]:
import subprocess
import platform

file_name = 'reverse_string.cpp'
input_file_name = 'output_parsing.txt'

# Compilation command
compilation_command = f"g++ -o {file_name.split('.')[0]} -fprofile-arcs -ftest-coverage {file_name}"

# Executable name
executable_name = "./" +file_name.split('.')[0]

# Executing command
executing_command = f"./{executable_name} < {input_file_name}" if platform.system() != 'Windows' else f"{executable_name} < {input_file_name}"

# Coverage command
coverage_command = f"gcov {executable_name}"

In [ ]:
try:
    subprocess.run(compilation_command, shell=True, check=True)
    print("Compilation successful\n" + '-'*50)
except subprocess.CalledProcessError as e:
    print(f"Compilation failed with error: {e}")

try:
    result = subprocess.run(executing_command, shell=True, check=True, capture_output=True, text=True)
    print(result.stdout + "\n" + '-'*50)
except subprocess.CalledProcessError as e:
    print(f"Execution failed with error: {e}")
    print("Error Output:", e.stderr)

try:
    result = subprocess.run(coverage_command, shell=True, check=True, capture_output=True, text=True)
    print(result.stdout + '\n')
except subprocess.CalledProcessError as e:
    print(f"Coverage failed with error: {e}")
    print("Error Output:", e.stderr)

Compilation successful
--------------------------------------------------
olleH

--------------------------------------------------
File 'reverse_string.cpp'
Lines executed:93.75% of 16
Creating 'reverse_string.cpp.gcov'

File '/usr/include/c++/11/iostream'
No executable lines
Removing 'iostream.gcov'

Lines executed:93.75% of 16




In [ ]:
# Specify the file path
file_path = file_name + '.gcov'  # Replace with the actual path to your text file

# Open the file in read mode ('r' for reading)
with open(file_path, 'r') as file:  # Remove quotes around file_path
    # Read the contents of the file
    file_contents = file.read()

# Display or process the file contents
print(file_contents)

        -:    0:Source:reverse_string.cpp
        -:    0:Graph:./reverse_string.gcno
        -:    0:Data:./reverse_string.gcda
        -:    0:Runs:1
        -:    1:// Include the iostream header file
        -:    2:#include <iostream>
        -:    3:#include <cstring>
        -:    4:using namespace std;
        -:    5:
        -:    6:// A function definition for a function that reverses a string
        1:    7:char* reverse_string(char* str) {
        -:    8:    // Get the length of the string
        1:    9:    int len = strlen(str);
        -:   10:    // Allocate memory for the reversed string
        1:   11:    char* rev = (char*)malloc(len + 1);
        -:   12:    // Check if the memory allocation was successful
        1:   13:    if (rev == NULL) {
    #####:   14:    return NULL;
        -:   15:    }
        -:   16:    // Copy the characters from the original string to the reversed string in reverse order
        6:   17:    for (int i = 0; i < len; i++) {
     

In [ ]:
import re

# Use regular expression to find lines after "#####"
pattern = re.compile(r'#####:\s+(\d+):(.*?)\n')
matches = pattern.findall(file_contents)
# Print the result
# Print the result
for match in matches:
    line_number = match[0]
    content = match[1].strip()
    print(f"Line {line_number}: {content}")

Line 14: return NULL;


In [ ]:
!g++ --version

g++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

